In [1]:
import ipynb.fs.full.get_news_headlines as gethl
import ipynb.fs.full.write_data as gswd
import pandas
import json
import os

In [7]:
def main():
    try:
        hl = gethl.read_livemint()  #Get headlines
        if(not(hl.empty)):
            classified_hl = pandas.DataFrame(json.loads(gethl.classify_headlines(hl['title'].tolist())))  # converting to json the model reply from Gemini AI.
            classified_titles = hl.join(classified_hl, lsuffix='_orig', rsuffix='_copy')   #make one df given with surety that matching is happening on indexes because Title matching used to give missed matches.. 
            #classified_titles = classified_titles[classified_titles.classification]  #keeping only the titles with a analyst recommerndation here.
            gswd.write_headlines(classified_titles)  #write titles to a google file
        else:
            print('No Headlines Received')
    except Exception as e:
        print(e)
    return 0

In [8]:
if __name__ == '__main__':
    hl = main()